In [1]:
sc

In [2]:
sc.version

'1.6.0'

In [3]:
sc.appName

'pyspark-shell'

In [4]:
sc.applicationId

'local-1491128901054'

In [5]:
sc.defaultParallelism

16

In [6]:
sc.defaultMinPartitions

2

In [7]:
sc.environment

{'PYTHONHASHSEED': '0'}

In [8]:
sc.master

'local[*]'

In [9]:
sc.startTime

1491128897782

In [10]:
sc.sparkUser()

'notebook'

In [11]:
sc.pythonVer

'3.4'

In [12]:
sc.PACKAGE_EXTENSIONS

('.zip', '.egg', '.jar')

In [13]:
sc.pythonExec

'python'

In [15]:
rdd = sc.parallelize(['I', 'will', 'complete', 'learning', 'Spark'])
rdd1 = sc.parallelize(['using', 'Jupyter' , 'Notebook', 'in', '20', 'days'],8)

In [16]:
rdd

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:423

In [17]:
rdd1

ParallelCollectionRDD[1] at parallelize at PythonRDD.scala:423

In [18]:
rdd.getNumPartitions()

16

In [19]:
rdd1.getNumPartitions()

8

In [20]:
rdd.toDebugString()

b'(16) ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:423 []'

In [21]:
rdd1.toDebugString()

b'(8) ParallelCollectionRDD[1] at parallelize at PythonRDD.scala:423 []'

In [22]:
rdd2 = rdd1.coalesce(2)

In [23]:
rdd2.getNumPartitions()

2

# Lab 1 : MapR

In [24]:
# read auction data into an RDD
auctionRDD = sc.textFile("./data/MaprDataFiles/auctiondata.csv")

In [25]:
# explore data in the file by looking at first 10 lines using 'take' command 
auctionRDD.take(10)

['8213034705,95,2.927373,jake7870,0,95,117.5,xbox,3',
 '8213034705,115,2.943484,davidbresler2,1,95,117.5,xbox,3',
 '8213034705,100,2.951285,gladimacowgirl,58,95,117.5,xbox,3',
 '8213034705,117.5,2.998947,daysrus,10,95,117.5,xbox,3',
 '8213060420,2,0.065266,donnie4814,5,1,120,xbox,3',
 '8213060420,15.25,0.123218,myreeceyboy,52,1,120,xbox,3',
 '8213060420,3,0.186539,parakeet2004,5,1,120,xbox,3',
 '8213060420,10,0.18669,parakeet2004,5,1,120,xbox,3',
 '8213060420,24.99,0.187049,parakeet2004,5,1,120,xbox,3',
 '8213060420,20,0.249491,bluebubbles_1,25,1,120,xbox,3']

In [26]:
# if you see above output, we can observe that 
# 1. Output is a list of elements
# 2. each line in the file has become an element

# This is not usable for our analysis. We need to split each line into seperate fields based on comma 
# delimiter

auctionsRDD = auctionRDD.map(lambda line: line.split(","))

In [32]:
# lets take a look at data inside auctionsRDD
auctionsRDD.take(3)

[['8213034705', '95', '2.927373', 'jake7870', '0', '95', '117.5', 'xbox', '3'],
 ['8213034705',
  '115',
  '2.943484',
  'davidbresler2',
  '1',
  '95',
  '117.5',
  'xbox',
  '3'],
 ['8213034705',
  '100',
  '2.951285',
  'gladimacowgirl',
  '58',
  '95',
  '117.5',
  'xbox',
  '3']]

In [27]:
# If you see above output, it is a list of lists each list inside the main list represents a line in the file
# Each element inside the inner list represents a field or value

# Now, declare variables and assign positions so that we can refer various columns in the above RDD using names instead of numbers. 
# Ex: auctionsRDD[aucid] instead of auctionsRDD[0] 
# this approach is for best practice as it improves readablility.
aucid = 0
bid = 1
bidtime = 2
bidder = 3
bidderrate = 4
openbid = 5
price = 6
itemtype = 7
dtl = 8

#### Lets explore this dataset more !!

In [28]:
# 1. How do you see the first element of the auctionsRDD?
auctionsRDD.first()

['8213034705', '95', '2.927373', 'jake7870', '0', '95', '117.5', 'xbox', '3']

In [29]:
# 2. What do you use to see the first 5 elements of the RDD?
auctionsRDD.take(5)

[['8213034705', '95', '2.927373', 'jake7870', '0', '95', '117.5', 'xbox', '3'],
 ['8213034705',
  '115',
  '2.943484',
  'davidbresler2',
  '1',
  '95',
  '117.5',
  'xbox',
  '3'],
 ['8213034705',
  '100',
  '2.951285',
  'gladimacowgirl',
  '58',
  '95',
  '117.5',
  'xbox',
  '3'],
 ['8213034705',
  '117.5',
  '2.998947',
  'daysrus',
  '10',
  '95',
  '117.5',
  'xbox',
  '3'],
 ['8213060420', '2', '0.065266', 'donnie4814', '5', '1', '120', 'xbox', '3']]

In [30]:
# 3. What is the total number of bids?
totbids = auctionsRDD.count()
totbids

10654

In [31]:
# 4. What is the total number of distinct items that were auctioned?
totitems = auctionsRDD.map(lambda auction: auction[aucid]).distinct().count()
totitems

627

In [32]:
# 5. What is the total number of item types that were auctioned?
totitemtype = auctionsRDD.map(lambda auction:auction[itemtype]).distinct().count()
totitemtype

3

In [33]:
# 6. What is the total number of bids per item type?
bids_itemtype = auctionsRDD.map(lambda auction : (auction[itemtype], 1)).reduceByKey(lambda a,b:a+b)
bids_itemtype.collect()
#otherways
# totitemtype = auctionsRDD.map(lambda auction:(auction[itemtype],1)).countByKey()
# totitemtype = auctionsRDD.map(lambda auction:auction[itemtype]).countByValue()

[('xbox', 2784), ('palm', 5917), ('cartier', 1953)]

In [34]:
# 8. Across all auctioned items, what is the maximum number of bids?
maxbids = auctionsRDD.map(lambda auction: (auction[aucid],1)) \
                            .reduceByKey(lambda x,y:x+y) \
                            .map(lambda x: x[1]).reduce(max)
maxbids        

75

In [35]:
# 9. Across all auctioned items, what is the minimum number of bids?
minbids = auctionsRDD.map(lambda auction: (auction[aucid],1)) \
                            .reduceByKey(lambda x,y:x+y) \
                            .map(lambda x: x[1]).reduce(min)
minbids        

1

In [36]:
# 10. What is the average number of bids per auction? Hint: totalbids/totalauctions
#totalbids = auctionsRDD.map(lambda auction: (auction[aucid],1)).reduceByKey(lambda x,y:x+y).map(lambda x: x[1]).sum()
avgbids = totbids/totitems      
avgbids

16.992025518341308

# Caching RDDs:

In [37]:
#7. Create an RDD that contains total bids for each auction.
bidsAuctionRDD = auctionsRDD.map(lambda auction: (auction[aucid],1)).reduceByKey(lambda x,y:x+y)
bidsAuctionRDD.persist()    

PythonRDD[38] at RDD at PythonRDD.scala:43

In [38]:
bidsAuctionRDD.take(5)

[('3024579522', 7),
 ('3018964794', 20),
 ('3014314236', 5),
 ('8214279576', 4),
 ('1643136423', 10)]

In [39]:
bidsAuctionRDD.toDebugString()

b'(2) PythonRDD[38] at RDD at PythonRDD.scala:43 [Memory Serialized 1x Replicated]\n |       CachedPartitions: 1; MemorySize: 4.7 KB; ExternalBlockStoreSize: 0.0 B; DiskSize: 0.0 B\n |  MapPartitionsRDD[37] at mapPartitions at PythonRDD.scala:374 [Memory Serialized 1x Replicated]\n |  ShuffledRDD[36] at partitionBy at NativeMethodAccessorImpl.java:-2 [Memory Serialized 1x Replicated]\n +-(2) PairwiseRDD[35] at reduceByKey at <ipython-input-37-0d8d74e87bfd>:2 [Memory Serialized 1x Replicated]\n    |  PythonRDD[34] at reduceByKey at <ipython-input-37-0d8d74e87bfd>:2 [Memory Serialized 1x Replicated]\n    |  MapPartitionsRDD[4] at textFile at NativeMethodAccessorImpl.java:-2 [Memory Serialized 1x Replicated]\n    |  ./data/MaprDataFiles/auctiondata.csv HadoopRDD[3] at textFile at NativeMethodAccessorImpl.java:-2 [Memory Serialized 1x Replicated]'

In [40]:
maxbids = bidsAuctionRDD.map(lambda bid: bid[1]).reduce(max)
maxbids
# maxbids = bidsAuctionRDD.map(lambda bid: bid[1]).max()

75

In [43]:
minbids = bidsAuctionRDD.map(lambda bid: bid[1]).reduce(min)
minbids
# minbids = bidsAuctionRDD.map(lambda bid: bid[1]).min()

1

In [46]:
auctionsRDD.filter(lambda x: '1639672910' in x).collect()

[['1639672910',
  '5200',
  '6.291875',
  'esmodeus',
  '0',
  '5000',
  '5400',
  'cartier',
  '7'],
 ['1639672910',
  '5100',
  '6.925520833',
  'akapson',
  '7',
  '5000',
  '5400',
  'cartier',
  '7'],
 ['1639672910',
  '5300',
  '6.926469907',
  'akapson',
  '7',
  '5000',
  '5400',
  'cartier',
  '7'],
 ['1639672910',
  '5400',
  '6.933402778',
  'esmodeus',
  '0',
  '5000',
  '5400',
  'cartier',
  '7']]

In [48]:
maxbidamt = auctionsRDD.map(lambda x: (x[0],x[1])).max(key = lambda x : float(x[1]))
maxbidamt

('1639672910', '5400')

In [49]:
# COntents of an RDD inside each partition

a = sc.parallelize(range(1, 100))
# Return an RDD created by coalescing all elements within each partition into a list.
a.glom().collect()

[[1, 2, 3, 4, 5, 6],
 [7, 8, 9, 10, 11, 12],
 [13, 14, 15, 16, 17, 18],
 [19, 20, 21, 22, 23, 24],
 [25, 26, 27, 28, 29, 30],
 [31, 32, 33, 34, 35, 36, 37],
 [38, 39, 40, 41, 42, 43],
 [44, 45, 46, 47, 48, 49],
 [50, 51, 52, 53, 54, 55],
 [56, 57, 58, 59, 60, 61],
 [62, 63, 64, 65, 66, 67, 68],
 [69, 70, 71, 72, 73, 74],
 [75, 76, 77, 78, 79, 80],
 [81, 82, 83, 84, 85, 86],
 [87, 88, 89, 90, 91, 92],
 [93, 94, 95, 96, 97, 98, 99]]

## Lab 2

In [50]:
sqlContext

In [51]:
type(sqlContext)

pyspark.sql.context.HiveContext

In [52]:
from pyspark.sql import Row

In [53]:
# Load a text file and inferring the Schema Using Reflection
inputRDD = sc.textFile("./data/MaprDataFiles/auctiondata.csv").map(lambda lines: lines.split(","))
inputRDD.take(5)

[['8213034705', '95', '2.927373', 'jake7870', '0', '95', '117.5', 'xbox', '3'],
 ['8213034705',
  '115',
  '2.943484',
  'davidbresler2',
  '1',
  '95',
  '117.5',
  'xbox',
  '3'],
 ['8213034705',
  '100',
  '2.951285',
  'gladimacowgirl',
  '58',
  '95',
  '117.5',
  'xbox',
  '3'],
 ['8213034705',
  '117.5',
  '2.998947',
  'daysrus',
  '10',
  '95',
  '117.5',
  'xbox',
  '3'],
 ['8213060420', '2', '0.065266', 'donnie4814', '5', '1', '120', 'xbox', '3']]

In [55]:
# Parse each line and create a row with proper datatypes
auctionsRDD = inputRDD.map(lambda line: Row(aucid = int(line[0]), bid = float(line[1]), bidtime = float(line[2]), \
                                            bidder = line[3], bidderate = int(line[4]), openbid = float(line[5]), \
                                            price = float(line[6]), itemtype = line[7], dtl = int(line[8])))


In [56]:
type(inputRDD)

pyspark.rdd.PipelinedRDD

In [57]:
type(auctionsRDD)

pyspark.rdd.PipelinedRDD

In [63]:
auctionsRDD.toDebugString()

b'(2) PythonRDD[60] at RDD at PythonRDD.scala:43 []\n |  MapPartitionsRDD[52] at textFile at NativeMethodAccessorImpl.java:-2 []\n |  ./data/MaprDataFiles/auctiondata.csv HadoopRDD[51] at textFile at NativeMethodAccessorImpl.java:-2 []'

In [60]:
xboxRDD = auctionsRDD.map(lambda line : "xbox" in line[7])

In [62]:
xboxRDD.toDebugString()

b'(2) PythonRDD[59] at RDD at PythonRDD.scala:43 []\n |  MapPartitionsRDD[52] at textFile at NativeMethodAccessorImpl.java:-2 []\n |  ./data/MaprDataFiles/auctiondata.csv HadoopRDD[51] at textFile at NativeMethodAccessorImpl.java:-2 []'

In [58]:
# create a DataFrame for above RDD
auctionsDF = sqlContext.createDataFrame(auctionsRDD)

In [64]:
auctionsDF.show()

+----------+-----+--------------+---------+--------+---+--------+-------+-----+
|     aucid|  bid|        bidder|bidderate| bidtime|dtl|itemtype|openbid|price|
+----------+-----+--------------+---------+--------+---+--------+-------+-----+
|8213034705| 95.0|      jake7870|        0|2.927373|  3|    xbox|   95.0|117.5|
|8213034705|115.0| davidbresler2|        1|2.943484|  3|    xbox|   95.0|117.5|
|8213034705|100.0|gladimacowgirl|       58|2.951285|  3|    xbox|   95.0|117.5|
|8213034705|117.5|       daysrus|       10|2.998947|  3|    xbox|   95.0|117.5|
|8213060420|  2.0|    donnie4814|        5|0.065266|  3|    xbox|    1.0|120.0|
|8213060420|15.25|   myreeceyboy|       52|0.123218|  3|    xbox|    1.0|120.0|
|8213060420|  3.0|  parakeet2004|        5|0.186539|  3|    xbox|    1.0|120.0|
|8213060420| 10.0|  parakeet2004|        5| 0.18669|  3|    xbox|    1.0|120.0|
|8213060420|24.99|  parakeet2004|        5|0.187049|  3|    xbox|    1.0|120.0|
|8213060420| 20.0| bluebubbles_1|       

In [65]:
auctionsDF.printSchema()

root
 |-- aucid: long (nullable = true)
 |-- bid: double (nullable = true)
 |-- bidder: string (nullable = true)
 |-- bidderate: long (nullable = true)
 |-- bidtime: double (nullable = true)
 |-- dtl: long (nullable = true)
 |-- itemtype: string (nullable = true)
 |-- openbid: double (nullable = true)
 |-- price: double (nullable = true)



In [69]:
# Register dataframe as a table
auctionsDF.registerTempTable("auctionTbl")

In [67]:
a

PythonRDD[62] at RDD at PythonRDD.scala:43

In [70]:
type(auctionTbl)

NameError: name 'auctionTbl' is not defined

In [73]:
sqlContext.sql("show tables").show()

+----------+-----------+
| tableName|isTemporary|
+----------+-----------+
|auctiontbl|       true|
+----------+-----------+



In [74]:
auctionsDF.count()

10654

In [75]:
auctionsDF.cache()

DataFrame[aucid: bigint, bid: double, bidder: string, bidderate: bigint, bidtime: double, dtl: bigint, itemtype: string, openbid: double, price: double]

In [76]:
auctionsDF.describe()

DataFrame[summary: string, aucid: string, bid: string, bidderate: string, bidtime: string, dtl: string, openbid: string, price: string]

In [77]:
auctionsDF.columns

['aucid',
 'bid',
 'bidder',
 'bidderate',
 'bidtime',
 'dtl',
 'itemtype',
 'openbid',
 'price']

In [78]:
auctionsDF.groupby("itemtype").count().show()

+--------+-----+
|itemtype|count|
+--------+-----+
|    xbox| 2784|
|    palm| 5917|
| cartier| 1953|
+--------+-----+



In [79]:
auctionsDF.select("itemtype").distinct().show()

+--------+
|itemtype|
+--------+
|    xbox|
|    palm|
| cartier|
+--------+



## Lets answer few questions about data in this dataframe

In [80]:
#1. What is the total number of bids?
auctionsDF.count()

10654

In [81]:
#2. What is the number of distinct auctions?
auctionsDF.select("aucid").distinct().count()

627

In [82]:
#3. What is the number of distinct itemtypes?
auctionsDF.select("itemtype").distinct().count()

3

In [83]:
#4. We would like a count of bids per auction and the item type (as shown below). How would you do this?
auctionsDF.groupBy("itemtype", "aucid").count().show()

+--------+----------+-----+
|itemtype|     aucid|count|
+--------+----------+-----+
|    palm|3023389524|   16|
|    palm|3023746524|   34|
|    xbox|8214786481|   16|
|    palm|3017911925|    8|
| cartier|1642414724|   14|
| cartier|1644739924|   18|
|    palm|3017841326|   15|
|    palm|3024579326|   11|
|    xbox|8214418083|   12|
|    palm|3013839128|   22|
|    xbox|8211763485|   31|
|    xbox|8212610485|   10|
|    xbox|8212755285|   26|
| cartier|1641628327|    7|
| cartier|1640936328|   22|
|    palm|3019224930|   30|
|    palm|3018457930|   29|
|    palm|3020454530|   21|
|    xbox|8212966887|   12|
|    xbox|8214767887|    9|
+--------+----------+-----+
only showing top 20 rows



In [84]:
# 5. For each auction item and item type, we want the max, min and average number of bids.
import pyspark.sql.functions as func
auctionsDF.groupBy("itemtype","aucid").count().agg(func.max("count").alias("Max"), func.min("count").alias("Min"), func.avg("count").alias("Count")).show()

+---+---+------------------+
|Max|Min|             Count|
+---+---+------------------+
| 75|  1|16.992025518341308|
+---+---+------------------+



In [85]:
# 6. For each auction item and item type, we want the max, min and average of a bid:
auctionsDF.groupBy("itemtype","aucid").agg(func.max("bid").alias("MaximumBid"), func.min("bid").alias("MinimumBid"), func.avg("bid").alias("AvgBid")).show()

+--------+----------+----------+----------+------------------+
|itemtype|     aucid|MaximumBid|MinimumBid|            AvgBid|
+--------+----------+----------+----------+------------------+
|    palm|3023389524|     212.5|      9.99|         111.84375|
|    palm|3023746524|     177.0|       1.0| 77.47323529411766|
|    xbox|8214786481|     120.0|      44.0|           86.3125|
|    palm|3017911925|     290.0|     225.0|             257.5|
| cartier|1642414724|     510.0|      12.0|262.57142857142856|
| cartier|1644739924|     455.0|       1.0|247.73611111111111|
|    palm|3017841326|     238.0|     152.5|197.84466666666665|
|    palm|3024579326|     237.5|     190.0| 219.3172727272727|
|    xbox|8214418083|     140.5|      75.0|113.71833333333332|
|    palm|3013839128|    251.11|      30.0| 133.8577272727273|
|    xbox|8211763485|     139.3|      10.0| 67.37806451612903|
|    xbox|8212610485|      33.0|       1.0|              21.8|
|    xbox|8212755285|     182.5|      10.0|112.21346153

In [86]:
# 7. What is the number of auctions with final price greater than 200?
auctionsDF.filter(auctionsDF.price > 200).count()

7685

In [88]:
xboxes = sqlContext.sql("select * from auctiontbl where itemtype = 'xbox'")
xboxes.show()

+----------+-----+--------------+---------+--------+---+--------+-------+-----+
|     aucid|  bid|        bidder|bidderate| bidtime|dtl|itemtype|openbid|price|
+----------+-----+--------------+---------+--------+---+--------+-------+-----+
|8213034705| 95.0|      jake7870|        0|2.927373|  3|    xbox|   95.0|117.5|
|8213034705|115.0| davidbresler2|        1|2.943484|  3|    xbox|   95.0|117.5|
|8213034705|100.0|gladimacowgirl|       58|2.951285|  3|    xbox|   95.0|117.5|
|8213034705|117.5|       daysrus|       10|2.998947|  3|    xbox|   95.0|117.5|
|8213060420|  2.0|    donnie4814|        5|0.065266|  3|    xbox|    1.0|120.0|
|8213060420|15.25|   myreeceyboy|       52|0.123218|  3|    xbox|    1.0|120.0|
|8213060420|  3.0|  parakeet2004|        5|0.186539|  3|    xbox|    1.0|120.0|
|8213060420| 10.0|  parakeet2004|        5| 0.18669|  3|    xbox|    1.0|120.0|
|8213060420|24.99|  parakeet2004|        5|0.187049|  3|    xbox|    1.0|120.0|
|8213060420| 20.0| bluebubbles_1|       

In [178]:
auctionsDF.select(auctionsDF.aucid.alias("Auction ID"),auctionsDF.bid ).filter(auctionsDF.bid > 200).show()

+----------+------+
|Auction ID|   bid|
+----------+------+
|8213935134| 202.5|
|8213935134| 205.0|
|8213935134| 207.5|
|8212830525|386.87|
|8212830525| 203.0|
|8212830525| 210.0|
|8212830525| 216.0|
|8212830525| 222.0|
|8212830525| 228.0|
|8212830525| 239.0|
|8212830525| 245.0|
|8212830525| 250.0|
|8212830525| 260.0|
|8212830525| 275.0|
|8212830525| 285.0|
|8212830525| 300.0|
|8212830525| 350.0|
|8212830525|403.87|
|8212830525| 375.0|
|8212830525| 400.0|
+----------+------+
only showing top 20 rows



In [89]:
auctionsDF.crosstab("itemtype", "dtl").show()

+------------+----+---+----+
|itemtype_dtl|   3|  5|   7|
+------------+----+---+----+
|        xbox| 553|393|1838|
|        palm|1216|869|3832|
|     cartier| 250|355|1348|
+------------+----+---+----+



# Others

In [91]:
# Multiple Aggregate operations on the same column of a spark dataframe

from pyspark.sql.functions import mean, sum, max, col

df = sc.parallelize([(1, 3.0), (1, 3.0), (2, -5.0)]).toDF(["k", "v"])

groupBy = ["k"]
aggregate = ["v"] 
funs = [mean, sum, max]

exprs = [f(col(c)) for f in funs for c in aggregate]

# or equivalent df.groupby(groupBy).agg(*exprs)
df.groupby(*groupBy).agg(*exprs).show()

In [95]:
# 1. Random Data Generation
import pyspark.sql.functions as f
df = sqlContext.range(0,10)

In [96]:
type(df)

pyspark.sql.dataframe.DataFrame

In [97]:
# Generate two other columns using uniform distribution and normal distribution.
df.select("id", f.rand(seed=10).alias("uniform"), f.randn(seed=27).alias("normal")).show()

+---+-------------------+--------------------+
| id|            uniform|              normal|
+---+-------------------+--------------------+
|  0| 0.1982919638208397| 0.06157382353970104|
|  1|0.44292918521277047| -0.4798519469521663|
|  2| 0.2731073068483362|-0.15116027592854422|
|  3|0.27149331793166864|-0.18575112254167045|
|  4| 0.6037143578435027|   0.734722467897308|
|  5|0.20981311586756546|-0.40496097207840054|
|  6| 0.4013331117277633|-0.21535632933479554|
|  7| 0.8469085276976173|  0.4932212707592389|
|  8|  0.819164334221561|  -1.593640432116535|
|  9| 0.2626612522319226|  0.4085363219031828|
+---+-------------------+--------------------+



In [98]:
# A slightly different way to generate the two random columns
df = sqlContext.range(0, 10) \
               .withColumn('uniform', f.rand(seed=10)) \
               .withColumn('normal', f.randn(seed=27))

In [99]:
#2. Summary and Descriptive Statistics
df.describe().show()

+-------+------------------+------------------+--------------------+
|summary|                id|           uniform|              normal|
+-------+------------------+------------------+--------------------+
|  count|                10|                10|                  10|
|   mean|               4.5|0.4329416473403548|-0.13326671948526814|
| stddev|3.0276503540974917|0.2440977761617599|  0.6509693428098151|
|    min|                 0|0.1982919638208397|  -1.593640432116535|
|    max|                 9|0.8469085276976173|   0.734722467897308|
+-------+------------------+------------------+--------------------+



In [100]:
# describe on a subset of the columns
df.describe('uniform', 'normal').show()

+-------+------------------+--------------------+
|summary|           uniform|              normal|
+-------+------------------+--------------------+
|  count|                10|                  10|
|   mean|0.4329416473403548|-0.13326671948526814|
| stddev|0.2440977761617599|  0.6509693428098151|
|    min|0.1982919638208397|  -1.593640432116535|
|    max|0.8469085276976173|   0.734722467897308|
+-------+------------------+--------------------+



In [101]:
from pyspark.sql.functions import mean, min, max
df.select(mean('uniform'), min('uniform'), max('uniform')).show()

+------------------+------------------+------------------+
|      avg(uniform)|      min(uniform)|      max(uniform)|
+------------------+------------------+------------------+
|0.4329416473403548|0.1982919638208397|0.8469085276976173|
+------------------+------------------+------------------+



In [394]:
# 3. Sample covariance and correlation.
"""
Covariance is a measure of how two variables change with respect to each other. 
A positive number would mean that there is a tendency that as one variable increases, the other increases as well. 
A negative number would mean that as one variable increases, the other variable has a tendency to decrease. 
The sample covariance of two columns of a DataFrame can be calculated as follows:
"""
df.stat.cov('uniform', 'normal')

-0.024815950987931417

In [393]:
df.stat.cov('id', 'id')

9.166666666666664

In [397]:
# Correlation is a normalized measure of covariance that is easier to understand,
# as it provides quantitative measurements of the statistical dependence between two random variables.
df.stat.corr('uniform', 'normal')

-0.15617321967079692

In [398]:
df.stat.corr('id', 'id')

1.0

In [401]:
# Create a DataFrame with two columns (name, item) 
names = ["Alice", "Bob", "Mike"]
items = ["milk", "bread", "butter", "apples", "oranges"]
df = sqlContext.createDataFrame([(names[i % 3], items[i % 5]) for i in range(100)], ["name", "item"])

In [402]:
# Take a look at the first 10 rows.
df.show(10)

+-----+-------+
| name|   item|
+-----+-------+
|Alice|   milk|
|  Bob|  bread|
| Mike| butter|
|Alice| apples|
|  Bob|oranges|
| Mike|   milk|
|Alice|  bread|
|  Bob| butter|
| Mike| apples|
|Alice|oranges|
+-----+-------+
only showing top 10 rows



In [403]:
df.stat.crosstab("name", "item").show()

+---------+------+-------+------+----+-----+
|name_item|apples|oranges|butter|milk|bread|
+---------+------+-------+------+----+-----+
|      Bob|     6|      7|     7|   6|    7|
|     Mike|     7|      6|     7|   7|    6|
|    Alice|     7|      7|     6|   7|    7|
+---------+------+-------+------+----+-----+



In [417]:
freq = df.stat.freqItems(["name","item"],0.5)

In [418]:
freq.show()

+--------------+--------------+
|name_freqItems|item_freqItems|
+--------------+--------------+
|       [Alice]|     [oranges]|
+--------------+--------------+



# Supplimentary Mapr

In [102]:
sfpdRDD = sc.textFile("./data/MaprDataFiles/sfpd.csv") \
        .map(lambda line:line.split(",")) \
        .map(lambda i: Row(IncidentNum = int(i[0]), Category = i[1], Descript = i[2], DayOfWeek = i[3] \
                         , Date = i[4], Time = i[5], PdDistrict = i[6], Resolution = i[7] \
                         , Address = i[8], XCoord = i[9], YCoord = i[10], PdID = int(i[11])))
sfpdRDD.take(5)

[Row(Address='JACKSON_ST/POWELL_ST', Category='OTHER_OFFENSES', Date='7/9/15', DayOfWeek='Thursday', Descript='POSSESSION_OF_BURGLARY_TOOLS', IncidentNum=150599321, PdDistrict='CENTRAL', PdID=15059900000000, Resolution='ARREST/BOOKED', Time='23:45', XCoord='-122.4099006', YCoord='37.79561712'),
 Row(Address='300_Block_of_POWELL_ST', Category='LARCENY/THEFT', Date='7/9/15', DayOfWeek='Thursday', Descript='PETTY_THEFT_OF_PROPERTY', IncidentNum=156168837, PdDistrict='CENTRAL', PdID=15616900000000, Resolution='NONE', Time='23:45', XCoord='-122.4083843', YCoord='37.78782711'),
 Row(Address='JACKSON_ST/POWELL_ST', Category='OTHER_OFFENSES', Date='7/9/15', DayOfWeek='Thursday', Descript='DRIVERS_LICENSE/SUSPENDED_OR_REVOKED', IncidentNum=150599321, PdDistrict='CENTRAL', PdID=15059900000000, Resolution='ARREST/BOOKED', Time='23:45', XCoord='-122.4099006', YCoord='37.79561712'),
 Row(Address='MASONIC_AV/GOLDEN_GATE_AV', Category='OTHER_OFFENSES', Date='7/9/15', DayOfWeek='Thursday', Descript='D

In [103]:
sfpdDF = sqlContext.createDataFrame(sfpdRDD)

In [104]:
sfpdDF.printSchema()

root
 |-- Address: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- Descript: string (nullable = true)
 |-- IncidentNum: long (nullable = true)
 |-- PdDistrict: string (nullable = true)
 |-- PdID: long (nullable = true)
 |-- Resolution: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- XCoord: string (nullable = true)
 |-- YCoord: string (nullable = true)



In [105]:
sfpdDF.registerTempTable("sfpdtbl")

In [107]:
#Seeing the first few lines of the DataFrame

sfpdDF.head(5)

[Row(Address='JACKSON_ST/POWELL_ST', Category='OTHER_OFFENSES', Date='7/9/15', DayOfWeek='Thursday', Descript='POSSESSION_OF_BURGLARY_TOOLS', IncidentNum=150599321, PdDistrict='CENTRAL', PdID=15059900000000, Resolution='ARREST/BOOKED', Time='23:45', XCoord='-122.4099006', YCoord='37.79561712'),
 Row(Address='300_Block_of_POWELL_ST', Category='LARCENY/THEFT', Date='7/9/15', DayOfWeek='Thursday', Descript='PETTY_THEFT_OF_PROPERTY', IncidentNum=156168837, PdDistrict='CENTRAL', PdID=15616900000000, Resolution='NONE', Time='23:45', XCoord='-122.4083843', YCoord='37.78782711'),
 Row(Address='JACKSON_ST/POWELL_ST', Category='OTHER_OFFENSES', Date='7/9/15', DayOfWeek='Thursday', Descript='DRIVERS_LICENSE/SUSPENDED_OR_REVOKED', IncidentNum=150599321, PdDistrict='CENTRAL', PdID=15059900000000, Resolution='ARREST/BOOKED', Time='23:45', XCoord='-122.4099006', YCoord='37.79561712'),
 Row(Address='MASONIC_AV/GOLDEN_GATE_AV', Category='OTHER_OFFENSES', Date='7/9/15', DayOfWeek='Thursday', Descript='D

In [109]:
sfpdDF.show(5)

+--------------------+--------------+------+---------+--------------------+-----------+----------+--------------+-------------+-----+------------+-----------+
|             Address|      Category|  Date|DayOfWeek|            Descript|IncidentNum|PdDistrict|          PdID|   Resolution| Time|      XCoord|     YCoord|
+--------------------+--------------+------+---------+--------------------+-----------+----------+--------------+-------------+-----+------------+-----------+
|JACKSON_ST/POWELL_ST|OTHER_OFFENSES|7/9/15| Thursday|POSSESSION_OF_BUR...|  150599321|   CENTRAL|15059900000000|ARREST/BOOKED|23:45|-122.4099006|37.79561712|
|300_Block_of_POWE...| LARCENY/THEFT|7/9/15| Thursday|PETTY_THEFT_OF_PR...|  156168837|   CENTRAL|15616900000000|         NONE|23:45|-122.4083843|37.78782711|
|JACKSON_ST/POWELL_ST|OTHER_OFFENSES|7/9/15| Thursday|DRIVERS_LICENSE/S...|  150599321|   CENTRAL|15059900000000|ARREST/BOOKED|23:45|-122.4099006|37.79561712|
|MASONIC_AV/GOLDEN...|OTHER_OFFENSES|7/9/15| T

In [110]:
sfpdDF.take(5)

[Row(Address='JACKSON_ST/POWELL_ST', Category='OTHER_OFFENSES', Date='7/9/15', DayOfWeek='Thursday', Descript='POSSESSION_OF_BURGLARY_TOOLS', IncidentNum=150599321, PdDistrict='CENTRAL', PdID=15059900000000, Resolution='ARREST/BOOKED', Time='23:45', XCoord='-122.4099006', YCoord='37.79561712'),
 Row(Address='300_Block_of_POWELL_ST', Category='LARCENY/THEFT', Date='7/9/15', DayOfWeek='Thursday', Descript='PETTY_THEFT_OF_PROPERTY', IncidentNum=156168837, PdDistrict='CENTRAL', PdID=15616900000000, Resolution='NONE', Time='23:45', XCoord='-122.4083843', YCoord='37.78782711'),
 Row(Address='JACKSON_ST/POWELL_ST', Category='OTHER_OFFENSES', Date='7/9/15', DayOfWeek='Thursday', Descript='DRIVERS_LICENSE/SUSPENDED_OR_REVOKED', IncidentNum=150599321, PdDistrict='CENTRAL', PdID=15059900000000, Resolution='ARREST/BOOKED', Time='23:45', XCoord='-122.4099006', YCoord='37.79561712'),
 Row(Address='MASONIC_AV/GOLDEN_GATE_AV', Category='OTHER_OFFENSES', Date='7/9/15', DayOfWeek='Thursday', Descript='D

In [112]:
sqlContext.sql("select * from sfpdtbl limit 5").show()

+--------------------+--------------+------+---------+--------------------+-----------+----------+--------------+-------------+-----+------------+-----------+
|             Address|      Category|  Date|DayOfWeek|            Descript|IncidentNum|PdDistrict|          PdID|   Resolution| Time|      XCoord|     YCoord|
+--------------------+--------------+------+---------+--------------------+-----------+----------+--------------+-------------+-----+------------+-----------+
|JACKSON_ST/POWELL_ST|OTHER_OFFENSES|7/9/15| Thursday|POSSESSION_OF_BUR...|  150599321|   CENTRAL|15059900000000|ARREST/BOOKED|23:45|-122.4099006|37.79561712|
|300_Block_of_POWE...| LARCENY/THEFT|7/9/15| Thursday|PETTY_THEFT_OF_PR...|  156168837|   CENTRAL|15616900000000|         NONE|23:45|-122.4083843|37.78782711|
|JACKSON_ST/POWELL_ST|OTHER_OFFENSES|7/9/15| Thursday|DRIVERS_LICENSE/S...|  150599321|   CENTRAL|15059900000000|ARREST/BOOKED|23:45|-122.4099006|37.79561712|
|MASONIC_AV/GOLDEN...|OTHER_OFFENSES|7/9/15| T

In [113]:
# Print the schema in a tree format
sfpdDF.printSchema()

root
 |-- Address: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- Descript: string (nullable = true)
 |-- IncidentNum: long (nullable = true)
 |-- PdDistrict: string (nullable = true)
 |-- PdID: long (nullable = true)
 |-- Resolution: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- XCoord: string (nullable = true)
 |-- YCoord: string (nullable = true)



In [114]:
# Get all the categories on incidents
sfpdDF.select("Category").distinct().show()

+--------------------+
|            Category|
+--------------------+
|      OTHER_OFFENSES|
|       VEHICLE_THEFT|
|             RUNAWAY|
|               FRAUD|
|            TRESPASS|
|           LOITERING|
|SEX_OFFENSES/NON_...|
|            BURGLARY|
|DRIVING_UNDER_THE...|
|       LARCENY/THEFT|
|  DISORDERLY_CONDUCT|
|      MISSING_PERSON|
|         WEAPON_LAWS|
|         LIQUOR_LAWS|
|             ROBBERY|
|   RECOVERED_VEHICLE|
|        PROSTITUTION|
|     STOLEN_PROPERTY|
|          KIDNAPPING|
|             SUICIDE|
+--------------------+
only showing top 20 rows



In [116]:
sqlContext.sql("""
               SELECT DISTINCT Category 
                 FROM sfpdtbl
               """).show()

+--------------------+
|            Category|
+--------------------+
|      OTHER_OFFENSES|
|       VEHICLE_THEFT|
|             RUNAWAY|
|               FRAUD|
|            TRESPASS|
|           LOITERING|
|SEX_OFFENSES/NON_...|
|            BURGLARY|
|DRIVING_UNDER_THE...|
|       LARCENY/THEFT|
|  DISORDERLY_CONDUCT|
|      MISSING_PERSON|
|         WEAPON_LAWS|
|         LIQUOR_LAWS|
|             ROBBERY|
|   RECOVERED_VEHICLE|
|        PROSTITUTION|
|     STOLEN_PROPERTY|
|          KIDNAPPING|
|             SUICIDE|
+--------------------+
only showing top 20 rows



In [117]:
sfpdDF.select(sfpdDF['Category']).distinct().show()

+--------------------+
|            Category|
+--------------------+
|      OTHER_OFFENSES|
|       VEHICLE_THEFT|
|             RUNAWAY|
|               FRAUD|
|            TRESPASS|
|           LOITERING|
|SEX_OFFENSES/NON_...|
|            BURGLARY|
|DRIVING_UNDER_THE...|
|       LARCENY/THEFT|
|  DISORDERLY_CONDUCT|
|      MISSING_PERSON|
|         WEAPON_LAWS|
|         LIQUOR_LAWS|
|             ROBBERY|
|   RECOVERED_VEHICLE|
|        PROSTITUTION|
|     STOLEN_PROPERTY|
|          KIDNAPPING|
|             SUICIDE|
+--------------------+
only showing top 20 rows



In [118]:
# Get all the districts
sfpdDF.select("PdDistrict").alias("District").distinct().show()

+----------+
|PdDistrict|
+----------+
| INGLESIDE|
|  RICHMOND|
|      PARK|
|  NORTHERN|
|   TARAVAL|
|   CENTRAL|
|   BAYVIEW|
|   MISSION|
|TENDERLOIN|
|  SOUTHERN|
+----------+



In [119]:
# Get all the incidents by category and sort by category desc
sfpdDF.select("Category", "IncidentNum").sort(sfpdDF['Category'].desc()).show()

+-----------+-----------+
|   Category|IncidentNum|
+-----------+-----------+
|WEAPON_LAWS|  140274812|
|WEAPON_LAWS|  140262180|
|WEAPON_LAWS|  140274715|
|WEAPON_LAWS|  140275898|
|WEAPON_LAWS|  140274561|
|WEAPON_LAWS|  140268241|
|WEAPON_LAWS|  140263570|
|WEAPON_LAWS|  140268213|
|WEAPON_LAWS|  140272139|
|WEAPON_LAWS|  140268213|
|WEAPON_LAWS|  140272117|
|WEAPON_LAWS|  140267390|
|WEAPON_LAWS|  140272048|
|WEAPON_LAWS|  140264540|
|WEAPON_LAWS|  140269045|
|WEAPON_LAWS|  140267083|
|WEAPON_LAWS|  140277333|
|WEAPON_LAWS|  140266637|
|WEAPON_LAWS|  140272117|
|WEAPON_LAWS|  140266079|
+-----------+-----------+
only showing top 20 rows



In [121]:
sqlContext.sql("SELECT Category, IncidentNum from SFPDtbl order by Category").show()

+--------+-----------+
|Category|IncidentNum|
+--------+-----------+
|   ARSON|  140269186|
|   ARSON|  140149726|
|   ARSON|  140267130|
|   ARSON|  140206984|
|   ARSON|  140187710|
|   ARSON|  140275810|
|   ARSON|  140187005|
|   ARSON|  140256230|
|   ARSON|  140172197|
|   ARSON|  140241394|
|   ARSON|  140169376|
|   ARSON|  140226578|
|   ARSON|  140168475|
|   ARSON|  140229362|
|   ARSON|  140166855|
|   ARSON|  140268188|
|   ARSON|  140161786|
|   ARSON|  140234719|
|   ARSON|  140158783|
|   ARSON|  140155632|
+--------+-----------+
only showing top 20 rows



In [469]:
from pyspark.sql.functions import col
sfpdDF.select("Category", "IncidentNum").sort(col("Category").desc()).show()

+-----------+-----------+
|   Category|IncidentNum|
+-----------+-----------+
|WEAPON_LAWS|  150592228|
|WEAPON_LAWS|  150581732|
|WEAPON_LAWS|  150592228|
|WEAPON_LAWS|  150598652|
|WEAPON_LAWS|  150592228|
|WEAPON_LAWS|  150598351|
|WEAPON_LAWS|  150588926|
|WEAPON_LAWS|  150583368|
|WEAPON_LAWS|  150583368|
|WEAPON_LAWS|  150597626|
|WEAPON_LAWS|  150583368|
|WEAPON_LAWS|  150594133|
|WEAPON_LAWS|  150583368|
|WEAPON_LAWS|  150586839|
|WEAPON_LAWS|  150583368|
|WEAPON_LAWS|  150588744|
|WEAPON_LAWS|  150583211|
|WEAPON_LAWS|  150583681|
|WEAPON_LAWS|  150583017|
|WEAPON_LAWS|  150598674|
+-----------+-----------+
only showing top 20 rows



In [123]:
# Counts of all the incidents by category and district
sfpdDF.crosstab("Category", "PdDistrict").show()

+--------------------+--------+-------+-------+-------+----+--------+---------+----------+--------+-------+
| Category_PdDistrict|SOUTHERN|MISSION|BAYVIEW|TARAVAL|PARK|NORTHERN|INGLESIDE|TENDERLOIN|RICHMOND|CENTRAL|
+--------------------+--------+-------+-------+-------+----+--------+---------+----------+--------+-------+
|      MISSING_PERSON|    1764|   1739|   1469|   1191|1370|     826|     1430|       448|     584|    739|
|           VANDALISM|    3109|   2045|   2008|   1653|1042|    2081|     1908|       787|    1189|   2165|
|                TREA|       1|      1|      1|      2|   0|       1|        0|         0|       0|      0|
|         LIQUOR_LAWS|      93|     86|     43|     47|  36|      31|       26|        70|      28|     34|
|      SUSPICIOUS_OCC|    2364|   1755|   1675|   1109| 668|    1410|     1255|      1225|     929|   1269|
|          BAD CHECKS|      15|      8|      5|     10|   2|      10|        3|         1|       6|      9|
|PORNOGRAPHY/OBSCE...|      

In [124]:
sfpdDF.crosstab("Category", "DayOfWeek").show()

+--------------------+--------+---------+------+------+--------+------+-------+
|  Category_DayOfWeek|Thursday|Wednesday|Friday|Monday|Saturday|Sunday|Tuesday|
+--------------------+--------+---------+------+------+--------+------+-------+
|      MISSING_PERSON|    1587|     1665|  2100|  1607|    1645|  1343|   1613|
|           VANDALISM|    2394|     2426|  2804|  2409|    2904|  2674|   2376|
|                TREA|       0|        1|     0|     3|       1|     0|      1|
|         LIQUOR_LAWS|      70|       74|    93|    53|      76|    66|     62|
|      SUSPICIOUS_OCC|    1999|     2065|  2164|  1889|    1786|  1765|   1991|
|          BAD CHECKS|      16|       10|    10|    18|       3|     3|      9|
|PORNOGRAPHY/OBSCE...|       0|        2|     1|     2|       1|     1|      3|
|             BRIBERY|      18|       29|    26|    25|      27|    21|     13|
|            TRESPASS|     436|      433|   417|   430|     388|   384|    442|
|   RECOVERED_VEHICLE|     121|      119

In [125]:
sfpdDF.crosstab("PdDistrict","DayOfWeek").show()

+--------------------+---------+------+------+--------+-------+------+--------+
|PdDistrict_DayOfWeek|Wednesday|Monday|Friday|Saturday|Tuesday|Sunday|Thursday|
+--------------------+---------+------+------+--------+-------+------+--------+
|            SOUTHERN|    10353|  9905| 11608|   11093|  10276|  9619|   10454|
|           INGLESIDE|     4819|  4770|  5026|    4635|   4815|  4344|    4750|
|          TENDERLOIN|     4597|  4341|  4200|    4169|   4457|  3916|    4494|
|             MISSION|     7129|  6938|  7606|    7251|   7103|  7053|    7084|
|             TARAVAL|     4137|  4060|  4311|    3751|   4012|  3462|    3737|
|            RICHMOND|     3123|  3004|  3282|    3007|   2950|  2843|    3012|
|            NORTHERN|     6463|  6235|  7363|    7230|   6399|  6613|    6574|
|                PARK|     3545|  3260|  3515|    3276|   3239|  3162|    3380|
|             CENTRAL|     5921|  5487|  6621|    6840|   5349|  5984|    5712|
|             BAYVIEW|     5355|  5037| 

In [126]:
sfpdDF.cube("PdDistrict", "DayOfWeek", "Category").count().dropna().sort(col("count").desc()).show()
# .sort(col("PdDistrict"), col("DayOfWeek")).show()

+----------+---------+-------------+-----+
|PdDistrict|DayOfWeek|     Category|count|
+----------+---------+-------------+-----+
|  SOUTHERN| Saturday|LARCENY/THEFT| 4107|
|  SOUTHERN|   Friday|LARCENY/THEFT| 3997|
|  SOUTHERN| Thursday|LARCENY/THEFT| 3401|
|  SOUTHERN|   Sunday|LARCENY/THEFT| 3365|
|  SOUTHERN|Wednesday|LARCENY/THEFT| 3321|
|  SOUTHERN|   Monday|LARCENY/THEFT| 3254|
|  SOUTHERN|  Tuesday|LARCENY/THEFT| 3208|
|  NORTHERN| Saturday|LARCENY/THEFT| 2818|
|  NORTHERN|   Friday|LARCENY/THEFT| 2575|
|   CENTRAL| Saturday|LARCENY/THEFT| 2322|
|  NORTHERN|   Sunday|LARCENY/THEFT| 2309|
|   CENTRAL|   Friday|LARCENY/THEFT| 2202|
|  NORTHERN| Thursday|LARCENY/THEFT| 2186|
|   CENTRAL|Wednesday|LARCENY/THEFT| 2128|
|  NORTHERN|Wednesday|LARCENY/THEFT| 2080|
|  NORTHERN|   Monday|LARCENY/THEFT| 2078|
|  NORTHERN|  Tuesday|LARCENY/THEFT| 2019|
|   CENTRAL|   Sunday|LARCENY/THEFT| 2009|
|   CENTRAL| Thursday|LARCENY/THEFT| 1988|
|   CENTRAL|  Tuesday|LARCENY/THEFT| 1873|
+----------

In [129]:
# how many categories are there?
sfpdDF.select("Category").distinct().count()

39

In [131]:
sqlContext.sql("SELECT distinct Category FROM sfpdtbl").show()

+--------------------+
|            Category|
+--------------------+
|      OTHER_OFFENSES|
|       VEHICLE_THEFT|
|             RUNAWAY|
|               FRAUD|
|            TRESPASS|
|           LOITERING|
|SEX_OFFENSES/NON_...|
|            BURGLARY|
|DRIVING_UNDER_THE...|
|       LARCENY/THEFT|
|  DISORDERLY_CONDUCT|
|      MISSING_PERSON|
|         WEAPON_LAWS|
|         LIQUOR_LAWS|
|             ROBBERY|
|   RECOVERED_VEHICLE|
|        PROSTITUTION|
|     STOLEN_PROPERTY|
|          KIDNAPPING|
|             SUICIDE|
+--------------------+
only showing top 20 rows



In [132]:
#What are the top 10 Resolutions ?
sqlContext.sql("SELECT resolution, count(1) as rescount from sfpdtbl group by resolution order by rescount desc limit 10").show()

+--------------------+--------+
|          resolution|rescount|
+--------------------+--------+
|                NONE|  243538|
|       ARREST/BOOKED|   86766|
|        ARREST/CITED|   22925|
|   PSYCHOPATHIC_CASE|    8344|
|             LOCATED|    6878|
|           UNFOUNDED|    4551|
|COMPLAINANT_REFUS...|    4215|
|     JUVENILE_BOOKED|    2381|
|EXCEPTIONAL_CLEAR...|    1134|
|      JUVENILE_CITED|    1039|
+--------------------+--------+



In [133]:
# What are the top 10 most incident Categories?
sqlContext.sql("SELECT category, count(1) as inccount from sfpdtbl group by category order by inccount desc limit 10").show()

+--------------+--------+
|      category|inccount|
+--------------+--------+
| LARCENY/THEFT|   96955|
|OTHER_OFFENSES|   50611|
|  NON-CRIMINAL|   50269|
|       ASSAULT|   31843|
|     VANDALISM|   17987|
| VEHICLE_THEFT|   17581|
|      WARRANTS|   17508|
|      BURGLARY|   15374|
| DRUG/NARCOTIC|   14300|
|SUSPICIOUS_OCC|   13659|
+--------------+--------+

